# Google Maps API requests

Google Maps module for Python (Help page) [here](https://googlemaps.github.io/google-maps-services-python/docs/2.2/#module-googlemaps)

*Google Developers page* for getting the API key [here](https://developers.google.com/maps/)

![](https://developers.google.com/_static/1c93cfc82f/images/redesign-14/logo-color-knockout.svg)

## Gist

Here we want to create a OD matrix with, for each O/D pair the duration of the trip and the distance as Google Maps would return it on a simple route research. We could assume then that these data are, on average, the actual duration and distance people spend from O to D.

## Steps
#### API keys

Instead of scrapping the webpage GoogleMaps (which Google does not allow: they will ban your IP adress if they realize you created a scrapping algorithm that block their servers).

So we must use their API service on [Google Developers webpage](https://developers.google.com/). 

First we need to get a API key for the API we want to use. Each Google Maps Web Service request requires an API key or client ID. API keys are freely available with a Google Account at https://developers.google.com/console. The type of API key you need is a Server key.

To get an API key:

Visit https://developers.google.com/console and log in with a Google Account.
+ Select one of your existing projects, or create a new project.
+ Enable the API(s) you want to use. The Python Client for Google Maps Services accesses the following APIs:
 + Directions API
 + Distance Matrix API
 + Elevation API
 + Geocoding API
 + Geolocation API
 + Places API
 + Roads API
 + Time Zone API
+ Create a new Server key.

The information provided by each key is given on both the google developer page and the help page (link given before).

#### Installation

Second you need to install the google map package for python on your machine. From a Jupyter Notebook this is you line you want:

```python
import pip
!pip install -U googlemaps
```

#### To know before creating your algorithm

Here the challenge is to build big data skims. For instance I had 200x200 OD matrix, assuming the duration of the trip depend on the direction of this trip, it is 40'000 API requests required to populate my OD mat. The problem is, if you use free Google Maps API requests, you have a daily threshold of request: 2500 requests per day and per API key, and you can have up to 12 keys on a single account ie. you can request max 30'000 information per day. So I needed to run my algorithm on two diferent days to populate my entire OD mat.

So I created 10 keys, and I loop over my API keys to not being blocked. You can also get a paying plan [here](https://developers.google.com/maps/pricing-and-plans/). Do the simultation, It's can be super expensive!

#### Go ahead!

You can now take a look to the script I created to populate two OD mat in the same api request. I used the Distance Matrix API, with mode "driving" and the desired departure time during morning peak of a random weekday "datetime.datetime(2017, 4, 3, 8, 30, 0, 0)". The Distance MAtrix API returns a lot of info (see help page) but I juste wanted (a) the distance and (b) the duration. Since I requestes at 0830 in the morning, I will get the expected trip duration in trafic conditions.

## Example of API requests GoogleMaps

You probably don't need to import all those guys but these are the regular packages I import to be sure I can manimulate all my data easily

In [3]:
%matplotlib inline

#import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import datetime
import googlemaps
import time

Here created a list of my 10 API keys:

In [15]:
APIs = ['AzaSyD8Q5hmpe-tJT5isTDkgYYrZI8NgFPB8OA',
   'AIzaSyDNkenufD2hVjB-107ocTmOSBURFznVW0',
   'AIzaSyAr4-I44Fa-6_i1VLb8YWWAspvq6VyfpI',
   'AIzaSyARy1J5bklWzo5n-f4v5JWLwTawwAyNcY',
   'AIzaSyDdDDI4tYoYEOqNlARFTzdMilMIaLt71o',
   'AIzaSyAlsP8elaG4LO8pKDYSK5mckvyAuH3Z_4',
   'AIzaSyCTwFxOclF_gAd3oHSjJV6QADouptnmFE',
   'AIzaSyB0gyr_sfKxXSe-E0-NEY6dZtCB2x4sKc',
   'AIzaSyD6Ttf-zZ8tpwBX2A5dWAXzKk-XWAnZ14',
   'AIzaSyDZNtnITEhr2z_HlBgRfcfsJlnc4KNb5k']

Here I load an empty matrix with the station as index and columns:

In [8]:
googleDurTraf = pd.read_csv('data/GoogleMapsScrap/google_empty.csv', index_col=[0])
googleDist = pd.read_csv('data/GoogleMapsScrap/google_empty.csv', index_col=[0])

In [9]:
googleDurTraf.head(2)

,Porter Square Station,MIT Stata Center at Vassar St / Main St,One Kendall Square at Hampshire St / Portland St,Lower Cambridgeport at Magazine St/Riverside Rd,Inman Square at Vellucci Plaza / Hampshire St,Central Square at Mass Ave / Essex St,Binney St / Sixth St,MIT at Mass Ave / Amherst St,Lafayette Square at Mass Ave / Main St / Columbia St,Lechmere Station at Cambridge St / First St,...,Watermark Seaport,Sidney Research Campus/ Erie Street at Waverly,Third at Binney,Central Square East Boston - Porter Street at London Street,Surface Rd at India St,Chinatown Gate Plaza,Copley Square - Dartmouth St at Boylston St,Congress St at North St,South Station - 700 Atlantic Ave,Rowes Wharf at Atlantic Ave
station_name,,,,,,,,,,,,,,,,,,,,,
Porter Square Station,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MIT Stata Center at Vassar St / Main St,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Since I can only have 2500 request per key, and I have 200 columns, I can do 12 rows per key

In [13]:
# Just a range that returns the number of stations I have:
lat = range(0,len(googleDurTraf))

Here is my loop. I obtain error at the end of each loop because I'm out of range either on my APY list or in my OD mat. You  could script exceptions, didn't took the time.

In [17]:
#help: https://googlemaps.github.io/google-maps-services-python/docs/2.2/#module-googlemaps
lat = rang
a = 0
b = a + 12

while b <= len(lat):
    for c in range(0,len(APIs)):
        gm = googlemaps.Client(key=APIs[c])#dur
        for i in range(a,b):
            for j in range(0,len(lat)):
                googleData = gm.distance_matrix(origins = origin[i], destinations = destination[j],  mode='driving',
                                departure_time= datetime.datetime(2017, 4, 3, 8, 30, 0, 0))
                #the api returns a dictionary, I just want a specific data in this dict:
                googleDur.iloc[i,j] = googleData['rows'][0]['elements'][0]['duration']['value']
                googleDurTraf.iloc[i,j] = googleData['rows'][0]['elements'][0]['duration_in_traffic']['value']
        # I save to_csv in each loop just to be sure to save some data... Probably make it slower. ~30minutes.     
        googleDur.to_csv(path_or_buf= 'data/GoogleMapsScrap/googleDur_driving.csv')
        googleDurTraf.to_csv(path_or_buf= 'data/GoogleMapsScrap/googleDur_driving_traffic.csv')
        if b == len(lat):
            b = len(lat) + 100
        else:
            a += 12
            b += 12
            if b > len(lat):
                b = len(lat)

IndexError: list index out of range

FYI, This is the dict you obtain from the API request:

In [111]:
googleData

{'destination_addresses': ['32 Albany St, Cambridge, MA 02139, USA'],
 'origin_addresses': ['55 Garden St, Cambridge, MA 02138, USA'],
 'rows': [{'elements': [{'distance': {'text': '4.1 km', 'value': 4088},
     'duration': {'text': '14 mins', 'value': 855},
     'duration_in_traffic': {'text': '17 mins', 'value': 1021},
     'status': 'OK'}]}],
 'status': 'OK'}